In [1]:
import site
site.addsitedir('Lib/site-packages')
import os
from glob import glob
from twelvelabs import TwelveLabs
from twelvelabs.models.task import Task
import requests
import ffmpeg
import m3u8_To_MP4
import json
from elevenlabs.client import ElevenLabs

In [2]:
KINDO_API_KEY = 

TL_API_KEY=
client = TwelveLabs(api_key=TL_API_KEY)


XI_API_KEY = 
elevenlabs = ElevenLabs(api_key=XI_API_KEY)


indexId =  #twelvelabs index for footage
writeIndexId =  #twelvelabs index for finished videos


TOTAL_VIDEO_DURATION = 0
INITIAL_PROMPT = "pumpkin"
TITLE = "pumpkin"

In [3]:
#sfx

def generate_sound_effect(text: str, output_path: str, duration):
   print("Generating sound effects...")

   result = elevenlabs.text_to_sound_effects.convert(
       text=text,
       duration_seconds=int(duration*.9),  # Optional, if not provided will automatically determine the correct length
       prompt_influence=0.3,  # Optional, if not provided will use the default value of 0.3
   )


   with open(output_path, "wb") as f:
       for chunk in result:
           f.write(chunk)


   print(f"Audio saved to {output_path}")


In [4]:
def getBestClip(page):
    clipDurations = []
    clipScores = []
    clipVideoIds = []
    clipUrls = []
    for clip in page:
        clipDurations.append(clip.end-clip.start)
        clipScores.append(clip.score)
        clipVideoIds.append(clip.video_id)
        clipVideo = client.index.video.retrieve(indexId, clip.video_id)
        clipUrls.append(clipVideo.hls.video_url)
    return findBestClip(clipDurations, clipScores, clipVideoIds,clipUrls)

def findBestClip(durations,scores,videoIds,urls):
    i = 0
    bestConfidence = 0;
    bestClipFound = videoIds[0];
    bestUrl = ""
    bestDuration = 0;
    for d in durations:
        if(int(d)>4 and int(d)<=22):
            if(int(scores[i])>bestConfidence and not checkIfUsedClip(videoIds[i])):
                bestDuration = d;
                bestConfidence = scores[i]
                bestClipFound = videoIds[i]
                bestUrl = urls[i]
        i = i+1
    print(str(bestClipFound) +" "+ str(bestConfidence) + bestUrl)
    m3u8_To_MP4.multithread_download(bestUrl)
    return bestClipFound, bestDuration;
def checkIfUsedClip(id):
    global usedVideoIds
    for i in usedVideoIds:
        if(id==i):
            print("tried to add repeated video")
            return True
    return False

def generateClip(textQuery: str, currentClipIndex: int):
    #Search Query
    global TOTAL_VIDEO_DURATION
    global TITLE
    global usedVideoIds
    search_results = client.search.query(index_id=indexId,
                                     query=textQuery,
                                     options=["visual"])
    coolId, coolDuration = getBestClip(search_results.data)
    usedVideoIds.append(coolId)
    TOTAL_VIDEO_DURATION = TOTAL_VIDEO_DURATION + int(coolDuration)

    #generatedNarration = stockClient.generate.text(video_id = coolId,
    #                          prompt = "Write a " + str(int(coolDuration*1.4)) + " word long frightening narration describing the subject of my video. Be a storyteller, be engaging."
    #)

    title = client.generate.text(video_id = coolId,
                          prompt = "Write ONE unique word for my video with NO SPECIAL CHARACTERS. NO QUOTATIONS"
    )

    generatedSoundPromptSubject = client.generate.text(video_id = coolId,
                          prompt = "Write an eerie ten word sound effect about only the MAIN SUBJECT of the video with NO SPECIAL CHARACTERS. NO COMMAS. NO VOICES. NO WHISPERS"
    )

    generatedSoundPromptBackground = client.generate.text(video_id = coolId,
                          prompt = "Write a scary ten word about only the BACKGROUND sound effect for the video. NO SPECIAL CHARACTERS. NO COMMAS"
    )

    nextQuery = client.generate.text(video_id = coolId,
                          prompt = "Write a two to three word prompt describing a loosely related topic to the video content. Go crazy. NO SPECIAL CHARACTERS. NO QUOTATIONS"
    )
    
    #generatedMovieSoundEffects = stockClient.generate.text(video_id = coolId,
    #                          prompt = "Write a six word long description of loud blaring instrumental movie trailer sound effects with NO SPECIAL CHARACTERS"
    #)
    
    os.rename("m3u8_To_MP4.mp4",f"{title.data}" +str(currentClipIndex)+".mp4")
    #print("Narration: "+f"{generatedNarration.data}")
    print("Subject: " + f"{generatedSoundPromptSubject.data}")
    print("Background: " + f"{generatedSoundPromptBackground.data}")
    print("Next query: " + f"{nextQuery.data}")
    #print("movieSFX: "+f"{generatedMovieSoundEffects.data}")
    generate_sound_effect(f"{generatedSoundPromptSubject.data}", f"{title.data}"+str(currentClipIndex)+"SubjectSFX.mp4",coolDuration)
    generate_sound_effect(f"{generatedSoundPromptBackground.data}", f"{title.data}"+str(currentClipIndex)+"BackgroundSFX.mp4",coolDuration)
    #generate_sound_effect(f"{generatedMovieSoundEffects.data}", f"{title.data}"+"MovieSFX.mp4",coolDuration)
    
    return f"{nextQuery.data}", f"{title.data}"+str(currentClipIndex)

In [5]:
usedVideoIds = []
secondPrompt, firstTitle = generateClip(INITIAL_PROMPT,0)
thirdPrompt, secondTitle = generateClip(secondPrompt,1)
fourthPrompt, thirdTitle = generateClip(thirdPrompt,2)

2024-10-14 08:14:03,407 | INFO | HTTP Request: POST https://api.twelvelabs.io/v1.2/search "HTTP/1.1 200 OK"
2024-10-14 08:14:03,547 | INFO | HTTP Request: GET https://api.twelvelabs.io/v1.2/indexes/670b8b58eaac6725641d69fe/videos/670b8bc29da39d4c05a1ef3d "HTTP/1.1 200 OK"
2024-10-14 08:14:03,692 | INFO | HTTP Request: GET https://api.twelvelabs.io/v1.2/indexes/670b8b58eaac6725641d69fe/videos/670b8bc29da39d4c05a1ef3e "HTTP/1.1 200 OK"
2024-10-14 08:14:03,840 | INFO | HTTP Request: GET https://api.twelvelabs.io/v1.2/indexes/670b8b58eaac6725641d69fe/videos/670b8c139da39d4c05a1ef5f "HTTP/1.1 200 OK"
2024-10-14 08:14:03,954 | INFO | HTTP Request: GET https://api.twelvelabs.io/v1.2/indexes/670b8b58eaac6725641d69fe/videos/670b8c13c0f5f53791d8eab5 "HTTP/1.1 200 OK"
2024-10-14 08:14:04,069 | INFO | HTTP Request: GET https://api.twelvelabs.io/v1.2/indexes/670b8b58eaac6725641d69fe/videos/670b8be49da39d4c05a1ef46 "HTTP/1.1 200 OK"
2024-10-14 08:14:04,191 | INFO | HTTP Request: GET https://api.twel

670b8bc29da39d4c05a1ef3d 84.06https://deuqpmn4rs7j5.cloudfront.net/670ac05d7c228a5c40d6b27b/670b8bc2e2f5d6a324a6bcc1/stream/97bbe162-4aec-4838-96b4-3f11e1d3eba3.m3u8

summary
m3u8_uri: https://deuqpmn4rs7j5.cloudfront.net/670ac05d7c228a5c40d6b27b/670b8bc2e2f5d6a324a6bcc1/stream/97bbe162-4aec-4838-96b4-3f11e1d3eba3.m3u8;
max_retry_times: 3;
tmp_dir: C:\Users\ivant\AppData\Local\Temp\m3u8_3825549662;
mp4_file_path: ./m3u8_To_MP4.mp4;



2024-10-14 08:14:05,167 | INFO | Choose the best bandwidth, which is 2417732
2024-10-14 08:14:05,167 | INFO | Best m3u8 uri is https://deuqpmn4rs7j5.cloudfront.net/670ac05d7c228a5c40d6b27b/670b8bc2e2f5d6a324a6bcc1/stream/97bbe162-4aec-4838-96b4-3f11e1d3eba3qvbr_fixedFR_qlauto_quantizationauto.m3u8


segment set: |##################################################| 100.0% downloaded segments successfully!


2024-10-14 08:14:05,999 | INFO | decrypt and dump segments...
2024-10-14 08:14:06,007 | INFO | merging segments...
2024-10-14 08:14:06,464 | INFO | download successfully！ take 1.64s,  average download speed is 1753.93KB/s
2024-10-14 08:14:12,947 | INFO | HTTP Request: POST https://api.twelvelabs.io/v1.2/generate "HTTP/1.1 200 OK"
2024-10-14 08:14:18,324 | INFO | HTTP Request: POST https://api.twelvelabs.io/v1.2/generate "HTTP/1.1 200 OK"
2024-10-14 08:14:22,514 | INFO | HTTP Request: POST https://api.twelvelabs.io/v1.2/generate "HTTP/1.1 200 OK"
2024-10-14 08:14:26,017 | INFO | HTTP Request: POST https://api.twelvelabs.io/v1.2/generate "HTTP/1.1 200 OK"


...

Subject: Haunting footsteps echo through foggy woods as eyes glow.
Background: Whispers of darkness fill the air with chilling dread.
Next query: Enchanted Night Adventure
Generating sound effects...


2024-10-14 08:15:53,279 | INFO | HTTP Request: POST https://api.elevenlabs.io/v1/sound-generation "HTTP/1.1 200 OK"


Audio saved to Mystical2SubjectSFX.mp4
Generating sound effects...


2024-10-14 08:16:02,873 | INFO | HTTP Request: POST https://api.elevenlabs.io/v1/sound-generation "HTTP/1.1 200 OK"


Audio saved to Mystical2BackgroundSFX.mp4


In [6]:
input1SubjectSFX = ffmpeg.input(firstTitle+"SubjectSFX.mp4")
input1BackgroundSFX = ffmpeg.input(firstTitle+"BackgroundSFX.mp4")


concatAudio1 = ffmpeg.filter([input1SubjectSFX, input1BackgroundSFX], 'amix', inputs=2,duration='longest')


input2SubjectSFX = ffmpeg.input(secondTitle+"SubjectSFX.mp4")
input2BackgroundSFX = ffmpeg.input(secondTitle+"BackgroundSFX.mp4")

concatAudio2 = ffmpeg.filter([input2SubjectSFX, input2BackgroundSFX], 'amix', inputs=2,duration='longest')

input3SubjectSFX = ffmpeg.input(thirdTitle+"SubjectSFX.mp4")
input3BackgroundSFX = ffmpeg.input(thirdTitle+"BackgroundSFX.mp4")

concatAudio3 = ffmpeg.filter([input3SubjectSFX, input3BackgroundSFX], 'amix', inputs=2,duration='longest')

audioMixed = ffmpeg.filter([concatAudio1,concatAudio2,concatAudio3], 'concat', n=3, a=1, v=0)
#Combine merged audio with the video
(
    ffmpeg
    .output(audioMixed,"processed_folder/wip"+TITLE+"AUDIOCONCAT.mp4", vcodec='copy', acodec='aac',strict='experimental')
    .run()
)




video1 = ffmpeg.input(firstTitle+".mp4")
video2 = ffmpeg.input(secondTitle+".mp4")
video3 = ffmpeg.input(thirdTitle+".mp4")


videoMixed = ffmpeg.concat(video1, video2, video3, v=1, a=0).node

(
    ffmpeg
    .output(videoMixed[0], "processed_folder/wip"+TITLE+"VideoPart.mp4")
    .run()
)

funkyAudio = ffmpeg.input("processed_folder/wip"+TITLE+"AUDIOCONCAT.mp4")
funkyVideo = ffmpeg.input("processed_folder/wip"+TITLE+"VideoPart.mp4")

# Combine merged audio with the video
try:
    (
        ffmpeg
        .output(funkyVideo, funkyAudio, "needsNarrationClips/"+TITLE+"NeedsNarrating.mp4", vcodec='copy', acodec='aac',format='mp4',strict='experimental')
        .run(capture_stdout=True, capture_stderr=True)
    )
except ffmpeg.Error as e:
    print("FFmpeg Error:", e.stderr.decode('utf-8'))
os.remove(firstTitle+".mp4")
os.remove(secondTitle+".mp4")
os.remove(thirdTitle+".mp4")
os.remove(firstTitle+"SubjectSFX.mp4")
os.remove(secondTitle+"SubjectSFX.mp4")
os.remove(thirdTitle+"SubjectSFX.mp4")
os.remove(firstTitle+"BackgroundSFX.mp4")
os.remove(secondTitle+"BackgroundSFX.mp4")
os.remove(thirdTitle+"BackgroundSFX.mp4")
os.remove("processed_folder/wip"+TITLE+"AUDIOCONCAT.mp4")
os.remove("processed_folder/wip"+TITLE+"VideoPart.mp4")

In [7]:
needNarratingId = ""

task = client.task.create(index_id=writeIndexId, file="needsNarrationClips/"+TITLE+"NeedsNarrating.mp4", language="en")
needNarratingId = task.video_id;

2024-10-14 08:16:10,669 | INFO | HTTP Request: POST https://api.twelvelabs.io/v1.2/tasks "HTTP/1.1 201 Created"
2024-10-14 08:16:10,791 | INFO | HTTP Request: GET https://api.twelvelabs.io/v1.2/tasks/670d35bae2f5d6a324a6c56b "HTTP/1.1 200 OK"


# WAIT HERE

# WAIT HERE

# WAIT HERE

In [1]:
generatedNarration = client.generate.text(video_id = needNarratingId,
                              prompt = "Write a bloodcurdling "+str((TOTAL_VIDEO_DURATION-TOTAL_VIDEO_DURATION%5)*2)+" word introduction to a horror film using the video as a basis. Begin by asking multiple questions. Refer to the audience as you. Find FIVE ordered unordinary specific details throughout the video and refer to them in the plot mysteriously. Use active voice. Be engaging, go wild! Ask questions to the audience. Announce a release date at the end." 
    )
print("narration: "+f"{generatedNarration.data}")

NameError: name 'client' is not defined

In [9]:
#kindo translation
MODEL_NAME = 'groq/llama-3.1-70b-versatile'
language = "JAPANESE"
text = f"{generatedNarration.data}"


# Define the endpoint and headers
url = "https://llm.kindo.ai/v1/chat/completions"
headers = {
    "api-key": KINDO_API_KEY,
    "content-type": "application/json"
}


# Define the payload
payload = {
    "model": MODEL_NAME,
    "messages": [{"role": "user", "content": f"Translate the following in {language} without the initial introductions (straight into the translation): {text}"}]
}


# Make the POST request
kindoResponse = requests.post(url, headers=headers, data=json.dumps(payload))
generatedKindoNarration = kindoResponse.json()['choices'][0]['message']['content']

# Print the response
print(generatedKindoNarration)

月の光で照らされた密度の高い森の中を、闇が這い詰める。ランターンを持っている人物が見えるが、その光そのものがほのぼのと浮かび上がる不思議な影を投げ出している。見知らぬ異形の生物が周囲を飛び回る。人物はこちらを向き、目の中に光るものが見え隠れする。ねじれた形で立っている木が前方に立っているが、その枝はむくりとこげつらにも見える。静寂が訪れるものの、悪夢はまだ終わっていない。


In [3]:
#narration
CHUNK_SIZE = 1024
VOICE_ID = "7fbQ7yJuEo56rYjrYaEh"  # ID of the voice model to use
TEXT_TO_SPEAK = f"{generatedNarration.data}"#generatedKindoNarration
OUTPUT_PATH = "needsNarrationClips/"+TITLE+"Narration.mp4"  # Path to save the output audio file
tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{VOICE_ID}/stream"

headers = {
   "Accept": "application/json",
   "xi-api-key": XI_API_KEY
}
data = {
   "text": TEXT_TO_SPEAK,
   "model_id": "eleven_multilingual_v2",
   "voice_settings": {
       "stability": 0.5,
       "similarity_boost": 0.8,
       "style": 0.0,
       "use_speaker_boost": True
   }
}

response = requests.post(tts_url, headers=headers, json=data, stream=True)
if response.ok:
   # Open the output file in write-binary mode
   with open(OUTPUT_PATH, "wb") as f:
       # Read the response in chunks and write to the file
       for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
           f.write(chunk)
   # Inform the user of success
   print("Audio stream saved successfully.")
else:
   # Print the error message if the request was not successful
   print(response.text)


NameError: name 'generatedNarration' is not defined

In [2]:
inputBaseVideo = ffmpeg.input("needsNarrationClips/"+TITLE+"NeedsNarrating.mp4")
inputNarration = ffmpeg.input(OUTPUT_PATH)

audioMixed = ffmpeg.filter([inputBaseVideo.audio,inputNarration.audio], 'amix', inputs=2,duration='longest')
# Combine merged audio with the video
try:
    (
        ffmpeg
        .output(inputBaseVideo.video, audioMixed, "FinalProduct/"+TITLE+"Video.mp4", vcodec='copy', acodec='aac',format='mp4',strict='experimental')
        .run(capture_stdout=True, capture_stderr=True)
    )
except ffmpeg.Error as e:
    print("FFmpeg Error:", e.stderr.decode('utf-8'))

NameError: name 'ffmpeg' is not defined